In [98]:
import pandas as pd
import glob
import os
import string
import numpy as np
import spacy
# gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# nltk
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

# 1 Import Data

## 1.1 Import Left-Wing Reddit Data

In [99]:
# get CSV file list from a foder
path = './radLeftSD/'
csv_files = glob.glob(path + "*.csv")

#read each CSV file into dataframe
df_list = (pd.read_csv(file) for file in csv_files)

leftwing_data = pd.concat(df_list, ignore_index=True)
leftwing_data

,Unnamed: 0,Title,Post Text,ID,Score,Total Comments,Post URL,Post Comments
0,0,if you have a moment can you please fill out t...,NaN,sy1c7u,31,21,https://firebasehostingproxy.page.link/1988868...,['this is the last full day the survey will be...
1,1,WE DECLARE 17 DAYS OF INTERNATIONAL CELEBRATIO...,\nTo counter HM King Charles III's declaration...,xawd3v,85,7,https://www.reddit.com/r/LateStageColonialism/...,['Does this include the Danish queen?']
2,2,YJA Star: We will win the battle and make the ...,NaN,11jw75t,41,1,https://i.redd.it/klrxl3urs3ma1.jpg,"['""We as YJA Star forces fighting for the nobl..."
3,3,Protesters in the Magway Region of Myanmar gua...,NaN,11jxcdf,4,0,https://i.redd.it/kvlggp7m24ma1.jpg,[]
4,4,Alkan: Guerrilla is always volunteering,NaN,11jztjh,1,0,https://www.nuceciwan119.xyz/en/2023/03/alkan-...,[]
...,...,...,...,...,...,...,...,...
5315,957,Donald Trump announcing that Gender would be d...,NaN,9q8qta,178,3,https://images.dailykos.com/images/276245/stor...,['Hope we can stop this asshole before he make...
5316,958,was told to post this here,NaN,9q7xy4,185,2,https://i.redd.it/9ilwo6qmamt11.jpg,"[""It would be hilarious if it wasn't true."", '..."
5317,959,Being trans is NOT a fetish,Fetishism embodies all of the worst aspects of...,9q9f88,41,2,https://www.reddit.com/r/LateStageGenderBinary...,['Wow. I like...\n\n&#x200B;\n\nHmmmm. Half ag...
5318,960,The State will not protect you. Buy a gun. Lea...,NaN,9q4wc4,187,17,https://twitter.com/LA_SocialistRA/status/1054...,['Also go beyond weapons (though I 100% agree ...


## 1.2 Import Right-Wing Patriots.Win Data

In [100]:
# get CSV file list from a foder
path = './altRightSD/'
csv_files = glob.glob(path + "*.csv")

#read each CSV file into dataframe
df_list = (pd.read_csv(file) for file in csv_files)

rightwing_data = pd.concat(df_list, ignore_index=True)
rightwing_data

,Unnamed: 0,Title,Post URL,Post Comments
0,0,\nTRUMP TWEET. DADDY SAYS BE IN DC ON JAN. 6TH\n,/p/11R4q2aptJ/trump-tweet-daddy-says-be-in-dc-/c/,"[""\nWell, shit. We've got marching orders, fre..."
1,1,\nThis is how we won\n,/p/GIEYMyjg/this-is-how-we-won/,"[""\nThey wouldn't have to censor us if we were..."
2,2,\nRaise your hand if you check this site as so...,/p/11Q8XPAOyg/raise-your-hand-if-you-check-thi/,"[""\nI've woken up at night to check TDW!\n"", '..."
3,3,\nUpvote if Fox News is dead to you too.\n,/p/11Q8O0hnaE/upvote-if-fox-news-is-dead-to-yo/,['\nAlex Jones just broke news Tucker is heavi...
4,4,\nNUMBER OF PEDES THAT DID NOT WATCH THE SHAM ...,/p/11S13uq5gt/number-of-pedes-that-did-not-wat/,"[""\nIt's on the front page of Twitch and chat ..."
...,...,...,...,...
719,236,\nTRUTH\n,/p/16aT36KnaQ/truth/c/,['\nSending this one to my 22 year old daughte...
720,237,\nSuspect in Shooting of Jews in L.A. Was Out ...,/p/16aANBPLca/suspect-in-shooting-of-jews-in-l/c/,['\nWho cares if they were Jews? If we can poi...
721,238,\n'I’m Begging You Not to Drink the Water’ in ...,/p/16aAN99eZo/im-begging-you-not-to-drink-the-/c/,['\nTruly is sickening the incompetence at our...
722,239,\n23.1 percent of Californians over the age of...,/p/16aAN7zKa9/231-percent-of-californians-over/c/,"[""\nThey can't read that sentence... because i..."


## 1.3 A brief note on how I fucked and am trying to unfuck the data
Essentially, the "Post Comments" column for each data frame was expected to consist of a clean list of posts for each entry. However, these lists only contain ONE BIG STRING consisting of all of the posts, which can be seen below: if you try to index into the first entry (e.g., the first list) you only get characters returned

In [101]:
rightwing_data["Post Comments"][0] # here is the first entry # 

'["\\nWell, shit. We\'ve got marching orders, frens.\\n", "\\nminecraft doesn\'t allow pickaxes on flying machines\\n", "\\nminecraft doesn\'t allow pickaxes on flying machines\\n", "\\nminecraft doesn\'t allow pickaxes on flying machines\\n", "\\nI\'m told DC is already full of hoes though.\\n", \'\\n[Dan Crenshaw has entered the chat]\\n\', \'\\n"I swear i never entered the chat my account was hacked"\\n\', \'\\nPatriots join and get in DONALD ARMY, state and nation chat – Organize, Go Together. We are organizing for Patriots to go to DC 1/6/21. We need local leaders to help recruit.\\nExamples of things being coordinated:\\nSharing of room/board and locations of safe places to book Traveling together Meetup Spots General Safety Events and locations Communication Ladies we can and will connect you with other women so you are safe.\\nInstructions (Do this from Desktop as it is too much for mobile):\\nNOTE: You should set up a private protonmail account with no personal information for

In [102]:
rightwing_data["Post Comments"][0][0] # indexing into this list, you get a char >:( # 

'['

In [103]:
leftwing_data["Post Comments"][0][0] # you can see this is also the case for leftwing data #

'['

## 1.4 Unfucking the data: preprocessing to try to regain the integrity of individual posts
As you can see, different posts are delimited by '\\n', so my hope is to try to split these large strings into separate posts this way

In [104]:
len(rightwing_data["Post Comments"][0].split('\\n'))

1897

Base preprocessing technique that can be applied to any given string

In [141]:
def base_preprocess(input_str):
    rez = []
    punctuation = string.punctuation + '…' + '—'
    filter_punct = str.maketrans('','', punctuation)
    stop_words = stopwords.words('english')
    stop_words.extend(["said", "i", "it", "you", "and","that", "u", "ur"])
    lemmatizer = WordNetLemmatizer()
    tokens = input_str.split()
    for token in tokens:
        token = token.lower().translate(filter_punct)
        if token not in stop_words:
            token = lemmatizer.lemmatize(token)
            rez.append(token)
    return rez

method that iterates through each df row and produces an actual list of comments for each df, that list being preprocessed

In [142]:
def process_df(df):
    dict_obj = {}
    for index, row in df.iterrows():
        key = ' '.join(base_preprocess(row["Title"]))
        dict_obj[key] = []
        strs = row["Post Comments"].split("\\n")
        for entry in strs:
            dict_obj[key].append(base_preprocess(entry))
    return dict_obj

## 1.5 Dictionaries for each title-comment section setup

In [143]:
rightwing_dict = process_df(rightwing_data)
print(list(rightwing_dict.keys())[:3],
      '\n',
      rightwing_dict[list(rightwing_dict.keys())[0]])

['trump tweet daddy say dc jan 6th', '', 'raise hand check site soon wake'] 
 [[''], ['well', 'shit', 'weve', 'got', 'marching', 'order', 'frens'], ['', ''], ['minecraft', 'doesnt', 'allow', 'pickax', 'flying', 'machine'], ['', ''], ['minecraft', 'doesnt', 'allow', 'pickax', 'flying', 'machine'], ['', ''], ['minecraft', 'doesnt', 'allow', 'pickax', 'flying', 'machine'], ['', ''], ['im', 'told', 'dc', 'already', 'full', 'hoe', 'though'], ['', ''], ['dan', 'crenshaw', 'entered', 'chat'], ['', ''], ['swear', 'never', 'entered', 'chat', 'account', 'hacked'], ['', ''], ['patriot', 'join', 'get', 'donald', 'army', 'state', 'nation', 'chat', '–', 'organize', 'go', 'together', 'organizing', 'patriot', 'go', 'dc', '1621', 'need', 'local', 'leader', 'help', 'recruit'], ['example', 'thing', 'coordinated'], ['sharing', 'roomboard', 'location', 'safe', 'place', 'book', 'traveling', 'together', 'meetup', 'spot', 'general', 'safety', 'event', 'location', 'communication', 'lady', 'connect', 'woman', '

In [144]:
leftwing_dict = process_df(leftwing_data)
print(list(leftwing_dict.keys())[:3],
      '\n',
      leftwing_dict[list(leftwing_dict.keys())[0]])

['moment please fill quick survey political influence', 'declare 17 day international celebration upon death queen elizabeth ii', 'yja star win battle make 21st century century woman'] 
 [['last', 'full', 'day', 'survey', 'open', 'filled', 'already', 'please', 'fill', 'arrived', 'view', 'traumabased', 'empathy', 'well', 'matriarchal', 'egalitarian', 'fertility', 'culture', 'people', 'equality', 'socialism', 'important', 'u', 'buildt', 'language', 'cultural', 'practice', 'know', 'dont', 'need', 'woke', 'colonizer', 'tell', 'u', 'organize', 'community', 'without', 'poverty', 'abuse', 'violence'], [], ['many', 'people', 'list', 'certainly', 'impacted', 'view', 'opposition', '', 'many', 'made', 'hate', 'authority', 'hierarchy', 'even', 'already', 'learning', 'narcissistic', 'megalomaniac', 'behaviour'], [], ['also', 'list', 'heavily', 'male', 'white', 'centric', 'men', 'many', 'privilege', 'teach', 'anything', 'importance', 'also', 'anyone', 'wish', 'add', 'please', 'let', 'know', 'definit

# 2 Topic Modeling: LDA

In [145]:
# vis imports
import pyLDAvis
import pyLDAvis.gensim

## 2.1 Producing word and freq vectors

In [146]:
# turn each posts dict item into a single, word vector
def gen_words(key, posts_dict):
    final = [word for word in key.split()]
    for entry in posts_dict[key]:
        for word in entry:
            if (word == ''):
                continue
            final.append(word)
    return (final)

In [147]:
def data_words(dictionary):
    foo = []
    for key in dictionary.keys():
        foo.append(gen_words(key, dictionary))
    return foo

rightwing_dw = data_words(rightwing_dict)
leftwing_dw = data_words(leftwing_dict)

In [148]:
def freq_vec(data_words):
    id2word = corpora.Dictionary(data_words)
    corpus = []
    for text in data_words:
        new = id2word.doc2bow(text)
        corpus.append(new)
    return id2word, corpus

## 2.2 Basic LDA for rightwing data

In [149]:
rightwing_id2word, rightwing_corpus = freq_vec(rightwing_dw)
print(rightwing_id2word[0][:5], 
     '\n',
     rightwing_corpus[:5])

1 
 [[(0, 6), (1, 3), (2, 11), (3, 1), (4, 3), (5, 2), (6, 1), (7, 1), (8, 1), (9, 2), (10, 2), (11, 1), (12, 1), (13, 1), (14, 5), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 5), (27, 3), (28, 3), (29, 1), (30, 1), (31, 1), (32, 1), (33, 3), (34, 3), (35, 1), (36, 2), (37, 4), (38, 1), (39, 6), (40, 4), (41, 2), (42, 1), (43, 1), (44, 3), (45, 1), (46, 4), (47, 3), (48, 5), (49, 2), (50, 1), (51, 15), (52, 1), (53, 20), (54, 1), (55, 2), (56, 3), (57, 2), (58, 1), (59, 4), (60, 3), (61, 1), (62, 4), (63, 9), (64, 3), (65, 3), (66, 2), (67, 5), (68, 2), (69, 1), (70, 1), (71, 2), (72, 2), (73, 8), (74, 1), (75, 10), (76, 6), (77, 1), (78, 7), (79, 6), (80, 2), (81, 1), (82, 2), (83, 1), (84, 12), (85, 3), (86, 1), (87, 1), (88, 1), (89, 2), (90, 2), (91, 1), (92, 2), (93, 2), (94, 4), (95, 2), (96, 2), (97, 7), (98, 3), (99, 1), (100, 6), (101, 1), (102, 1), (103, 7), (104, 1), (105, 10), (106, 4), (107, 1), (108, 2), (109, 1)

In [150]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=rightwing_corpus,
                                           id2word=rightwing_id2word,
                                           num_topics=3,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [151]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, rightwing_corpus, rightwing_id2word, mds="mmds", R=30)
vis

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.121201  0.102892       1        1  75.954115
1      0.136922 -0.099140       2        1  22.439402
0     -0.258123 -0.003752       3        1   1.606484, topic_info=          Term         Freq        Total Category  logprob  loglift
2162     trump  7286.000000  7286.000000  Default  30.0000  30.0000
675   election  3737.000000  3737.000000  Default  29.0000  29.0000
2234      vote  3712.000000  3712.000000  Default  28.0000  28.0000
1981     state  4512.000000  4512.000000  Default  27.0000  27.0000
1603     power  1416.000000  1416.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
1603     power    80.096983  1416.568595   Topic3  -5.5914   1.2584
391    chinese    51.460134   449.655327   Topic3  -6.0339   1.9634
649      drive    48.002747   329.427284   Topic3  -6.1034   2.2050
2204      used    50.542893  1424.585044   Topic3  -6.0519   0.7923
8973   natural    46.824656   270.302655   Topic3  -6.1283   2.3780

[264 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
58868      2  0.074599  001kwh
58868      3  0.895189  001kwh
58871      2  0.107114  1400mw
58871      3  0.856912  1400mw
11015      2  0.987620     270
...      ...       ...     ...
2333       3  0.010483    year
2346       1  0.936720   youre
2346       2  0.062383   youre
2346       3  0.000487   youre
30254      2  0.992745       ☑

[525 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

## 2.3 Basic LDA for leftwing data

In [152]:
leftwing_id2word, leftwing_corpus = freq_vec(leftwing_dw)

In [153]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=leftwing_corpus,
                                           id2word=leftwing_id2word,
                                           num_topics=3,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [154]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, leftwing_corpus, leftwing_id2word, mds="mmds", R=30)
vis

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.049107  0.246894       1        1  66.438016
1      0.271017 -0.063398       2        1  25.587035
2     -0.221910 -0.183495       3        1   7.974949, topic_info=          Term         Freq        Total Category  logprob  loglift
2290     trans  4588.000000  4588.000000  Default  30.0000  30.0000
78      please  6115.000000  6115.000000  Default  29.0000  29.0000
2284    gender  2535.000000  2535.000000  Default  28.0000  28.0000
581       rule  3050.000000  3050.000000  Default  27.0000  27.0000
164      woman  1778.000000  1778.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
4268  feminist   162.135175   210.730404   Topic3  -5.9529   2.2667
979       feel   250.200010  1051.330272   Topic3  -5.5191   1.0933
7271       gay   199.310989   974.277094   Topic3  -5.7465   0.9420
1553      left   189.970589   938.140731   Topic3  -5.7945   0.9318
448        way   186.706987  1828.714448   Topic3  -5.8118   0.2470

[183 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
536       1  0.012334    action
536       2  0.940222    action
536       3  0.047656    action
1891      1  0.999187  actually
1891      2  0.000717  actually
...     ...       ...       ...
2255      2  0.000846      work
227       1  0.999796     would
227       2  0.000379     would
751       1  0.998498      year
751       2  0.000816      year

[320 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

# 3 Advanced topic modeling with BERT